# Import Libraries

In [ ]:
# Import Required Libraries
import os # Saving of files
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical computing and array operations
import seaborn as sns  # Statistical data visualization
import matplotlib.pyplot as plt  # Graph plotting and visualization
from scipy.signal import butter, filtfilt, find_peaks  # Signal processing and filtering
from sklearn.metrics.pairwise import cosine_similarity  # Similarity measurement between features

# Filtering Functions

In [ ]:
# The code that acts as a bandpass filter, used to remove cable noise and motion artifacts
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

# Scale the signal between 1 and -1
def scale_signal(signal):
    min_signal, max_signal = signal.min(), signal.max()
    return 2 * (signal - min_signal) / (max_signal - min_signal) - 1

# Should be called simplify signal, removes any small noise and flattens the signal
def reduce_noise(signal, threshold):
    return np.where((np.abs(signal) < threshold) | (signal > 0), 0, signal)

# Designed to remove outliers however it seems to set the outlier to 0 which creates weird double troughs in the data
def remove_outliers(signal, threshold, outlier_lowerbound, outlier_upperbound):
    Q1 = np.percentile(signal, outlier_lowerbound)
    Q3 = np.percentile(signal, outlier_upperbound)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR

    median_value = np.median(signal[(signal >= lower_bound) & (signal <= upper_bound)])
    cleaned_signal = np.where((signal < lower_bound) | (signal > upper_bound), median_value, signal)

    return cleaned_signal

# Main Functions

In [ ]:
# In order to accuratly caclulate the duration the stablepoints must be snapped to the cloest peak.
def snap_stable_points_to_peaks(stable_points, bandpassed_signal, search_radius):
    snapped_points = []
    n = len(bandpassed_signal)

    for (start_idx, end_idx) in stable_points:
        start_left = max(0, start_idx - search_radius)
        start_right = start_idx
        if start_right - start_left > 0:
            local_max_start = np.argmax(bandpassed_signal[start_left:start_right+1])
            new_start_idx = start_left + local_max_start
        else:
            new_start_idx = start_idx
        end_left = end_idx
        end_right = min(n - 1, end_idx + search_radius)
        if end_right - end_left > 0:
            local_max_end = np.argmax(bandpassed_signal[end_left:end_right+1])
            new_end_idx = end_left + local_max_end
        else:
            new_end_idx = end_idx

        snapped_points.append((new_start_idx, new_end_idx))
    return snapped_points

# Finds all the troughs in the data, thier stable points and then searches for the peaks to snap the stable points too
def identify_troughs_and_stable_points(signal, threshold, time_array, bandpassed_signal, search_radius, trough_proximity_threshold):
    valid_troughs = []
    stable_indices = []
    amplitudes = []
    valid_indices = []

    troughs, _ = find_peaks(-signal, prominence=threshold)
    troughs = troughs[signal[troughs] < 0]

    for trough in troughs:
        before_idx_arr = np.where(signal[:trough] >= 0)[0]
        before_idx = before_idx_arr[-1] if len(before_idx_arr) > 0 else None

        after_idx_arr = np.where(signal[trough:] >= 0)[0]
        after_idx = after_idx_arr[0] + trough if len(after_idx_arr) > 0 else None

        if before_idx is not None and after_idx is not None:
            stable_indices.append((before_idx, after_idx))
            valid_troughs.append(trough)

    snapped_indices = snap_stable_points_to_peaks(stable_indices, bandpassed_signal, search_radius)

    for trough, (snap_before, snap_after) in zip(valid_troughs, snapped_indices):
        avg_stable = (bandpassed_signal[snap_before] + bandpassed_signal[snap_after]) / 2.0
        trough_value = bandpassed_signal[trough]
        amplitude = abs(avg_stable - trough_value)
        amplitudes.append(amplitude)

    trough_times = time_array[valid_troughs]

    sorted_order = np.argsort(trough_times)
    sorted_times = trough_times[sorted_order]
    sorted_amplitudes = np.array(amplitudes)[sorted_order]
    sorted_troughs = np.array(valid_troughs)[sorted_order]
    sorted_snapped = [snapped_indices[i] for i in sorted_order]

    i = 0
    while i < len(sorted_times):
        current_range = [i]
        j = i + 1
        while j < len(sorted_times) and (sorted_times[j] - sorted_times[i]) <= trough_proximity_threshold:
            current_range.append(j)
            j += 1
        max_amp_idx = current_range[np.argmax(np.abs(sorted_amplitudes[current_range]))]
        valid_indices.append(max_amp_idx)
        i = j

    valid_indices = np.array(valid_indices, dtype=int)
    trough_times = sorted_times[valid_indices]
    amplitudes = sorted_amplitudes[valid_indices]
    valid_troughs = sorted_troughs[valid_indices]
    snapped_indices = [sorted_snapped[i] for i in valid_indices]

    snapped_time_points = [(time_array[start], time_array[end]) for start, end in snapped_indices]

    return trough_times, snapped_time_points, amplitudes

# This is what actually calcualtes the values to be stored in the troughs
def analyze_troughs(signal, pb_dictionary):
    total_durations = []
    opening_durations = []
    closing_durations = []
    symmetries = []
    opening_velocities = []
    closing_velocities = []

    for i, trough_time in enumerate(pb_dictionary["trough_time"]):
        if (trough_time is None or np.isnan(trough_time) or
            i >= len(pb_dictionary["stable_points"]) or
            pb_dictionary["stable_points"][i] is None):
            print(f"Skipping index {i}: Invalid trough_time or no stable points.")
            total_durations.append(None)
            opening_durations.append(None)
            closing_durations.append(None)
            symmetries.append(None)
            opening_velocities.append(None)
            closing_velocities.append(None)
            continue

        before_idx, after_idx = pb_dictionary["stable_points"][i]
        opening_duration = abs(trough_time - before_idx)
        closing_duration = abs(after_idx - trough_time)
        total_duration = opening_duration + closing_duration

        if opening_duration == 0 and closing_duration == 0:
            symmetry_percentage = 100
        elif opening_duration == 0 or closing_duration == 0:
            symmetry_percentage = 0
        else:
            symmetry_percentage = (min(opening_duration, closing_duration) / max(opening_duration, closing_duration)) * 100

        if i < len(pb_dictionary["amplitude"]):
            trough_amplitude = pb_dictionary["amplitude"][i]
        else:
            print(f"Skipping index {i}: Trough amplitude is missing.")
            total_durations.append(None)
            opening_durations.append(None)
            closing_durations.append(None)
            symmetries.append(None)
            opening_velocities.append(None)
            closing_velocities.append(None)
            continue

        stable_before = signal[int(before_idx)]
        stable_after  = signal[int(after_idx)]
        opening_amp_diff = abs(stable_before - trough_amplitude)
        closing_amp_diff = abs(stable_after - trough_amplitude)

        opening_velocity = opening_amp_diff / opening_duration if opening_duration > 0 else None
        closing_velocity = closing_amp_diff / closing_duration if closing_duration > 0 else None

        opening_durations.append(opening_duration)
        closing_durations.append(closing_duration)
        total_durations.append(total_duration)
        symmetries.append(symmetry_percentage)
        opening_velocities.append(opening_velocity)
        closing_velocities.append(closing_velocity)

    return (total_durations, opening_durations, closing_durations, symmetries, opening_velocities, closing_velocities)

# This code then actually calcualtes the likeyhood of each trough being a blink
def identify_blinks(PB_Dictionary, min_blink_duration, max_blink_duration, blink_trough_threshold, symmetry_threshold, min_duration_percent, max_duration_percent, symmetry_percent, amplitude_percent):
    blink_likelyhoods = []
    is_blinks = []
    total_criteria = 100

    for i in range(len(PB_Dictionary["trough_time"])):
        criteria_matched = 0
        if (
            PB_Dictionary["duration_total"][i] is None or
            PB_Dictionary["symmetry"][i] is None or
            PB_Dictionary["amplitude"][i] is None
        ):
            blink_likelyhoods.append(0)
            is_blinks.append(0)
            continue

        total_duration = PB_Dictionary["duration_total"][i]
        symmetry = PB_Dictionary["symmetry"][i]
        amplitude = PB_Dictionary["amplitude"][i]

        if total_duration >= min_blink_duration:
            criteria_matched += min_duration_percent
        if total_duration <= max_blink_duration:
            criteria_matched += max_duration_percent
        if symmetry <= symmetry_threshold:
            criteria_matched += symmetry_percent
        if amplitude >= blink_trough_threshold:
            criteria_matched += amplitude_percent

        blink_likelihood = (criteria_matched / total_criteria) * 100
        is_blink = 1 if blink_likelihood >= 70 else 0

        blink_likelyhoods.append(blink_likelihood)
        is_blinks.append(is_blink)

    return blink_likelyhoods, is_blinks

# This checks which troughs are good by calculating new values
def third_pass(PB_Dictionary, amplitude_threshold, symmetry_threshold, outlier_method, likely_blink_percent, unlikey_blink_percent, likey_blink_factor, unlikey_blink_factor):
    tp_indices = [i for i, val in enumerate(PB_Dictionary["is_blink"]) if val == 1]
    features = ["amplitude", "symmetry", "blink_likelyhood"]

    if not tp_indices:
        print("No detected blinks to analyze.")
        return PB_Dictionary

    feature_matrix = {
        feature: [PB_Dictionary[feature][i] for i in tp_indices] for feature in features
    }

    def detect_outliers(data, method=outlier_method):
        data = np.array(data)
        if method == "iqr":
            q1, q3 = np.percentile(data, [25, 75])
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
        elif method == "zscore":
            mean = np.mean(data)
            std_dev = np.std(data)
            lower_bound = mean - 3 * std_dev
            upper_bound = mean + 3 * std_dev
        else:
            raise ValueError("Invalid outlier method. Use 'iqr' or 'zscore'.")
        return (data >= lower_bound) & (data <= upper_bound)

    valid_amplitude_mask = detect_outliers(feature_matrix["amplitude"], method=outlier_method)
    valid_symmetry_mask = detect_outliers(feature_matrix["symmetry"], method=outlier_method)
    valid_mask = valid_amplitude_mask & valid_symmetry_mask
    filtered_amplitude = np.array(feature_matrix["amplitude"])[valid_mask]
    filtered_symmetry = np.array(feature_matrix["symmetry"])[valid_mask]

    for i in tp_indices:
        amplitude = PB_Dictionary["amplitude"][i]
        symmetry = PB_Dictionary["symmetry"][i]
        likelihood = PB_Dictionary["blink_likelyhood"][i]

        likelihood_factor = 1.0
        if likelihood >= likely_blink_percent:
            likelihood_factor = likey_blink_factor
        elif likelihood <= unlikey_blink_percent:
            likelihood_factor = unlikey_blink_factor

        lower_amplitude = np.mean(filtered_amplitude) - amplitude_threshold * likelihood_factor
        upper_amplitude = np.mean(filtered_amplitude) + amplitude_threshold * likelihood_factor
        lower_symmetry = np.mean(filtered_symmetry) - symmetry_threshold * likelihood_factor
        upper_symmetry = np.mean(filtered_symmetry) + symmetry_threshold * likelihood_factor

        if (lower_amplitude <= amplitude <= upper_amplitude) and (lower_symmetry <= symmetry <= upper_symmetry):
            PB_Dictionary["is_blink"][i] = 1
        else:
            PB_Dictionary["is_blink"][i] = 0

    return PB_Dictionary

# Does a fatigue estimation based on if blinks are increasing, decreasing or staying the same
def track_fatigue(PB_Dictionary, time_window, time_increment, fatigue_increment):
    fatigue_score = 0
    past_avg_duration = 0
    past_avg_frequency = 0

    if "duration_total" not in PB_Dictionary or "trough_time" not in PB_Dictionary or "is_blink" not in PB_Dictionary:
        return fatigue_score

    blink_durations = [d for d, b in zip(PB_Dictionary["duration_total"], PB_Dictionary["is_blink"]) if b == 1]
    blink_timestamps = [t for t, b in zip(PB_Dictionary["trough_time"], PB_Dictionary["is_blink"]) if b == 1]

    if len(blink_durations) < 2:
        return fatigue_score

    max_time = max(blink_timestamps)

    for t in range(time_increment, int(max_time) + 1, time_increment):
        recent_durations = [d for time_val, d in zip(blink_timestamps, blink_durations) if t - time_window <= time_val <= t]
        avg_duration = np.mean(recent_durations) if recent_durations else past_avg_duration
        avg_frequency = len(recent_durations) / time_window if time_window > 0 else 0

        if avg_duration > past_avg_duration and avg_frequency > past_avg_frequency:
            fatigue_score += fatigue_increment

        past_avg_duration = avg_duration
        past_avg_frequency = avg_frequency

    return fatigue_score

# If the files are labeled this code checks if the guesses are correct
def validate_blinks(PB_Dictionary, blink_times, proximity_threshold, is_labeled=True):
    if 'blink_validation' not in PB_Dictionary or len(PB_Dictionary['blink_validation']) != len(PB_Dictionary['trough_time']):
        PB_Dictionary['blink_validation'] = [None] * len(PB_Dictionary['trough_time'])
    used_blink_times = set()

    for idx, trough_time in enumerate(PB_Dictionary['trough_time']):
        candidate_found = False
        matched_blink = None
        for blink_time in blink_times:
            if abs(trough_time - blink_time) <= proximity_threshold:
                candidate_found = True
                matched_blink = blink_time
                break
        if candidate_found:
            if PB_Dictionary['is_blink'][idx] == 1:
                PB_Dictionary['blink_validation'][idx] = 'TP'
            else:
                PB_Dictionary['blink_validation'][idx] = 'FN'
            used_blink_times.add(matched_blink)
        else:
            PB_Dictionary['blink_validation'][idx] = 'FP'

    for blink_time in blink_times:
        if blink_time not in used_blink_times and is_labeled:
            PB_Dictionary['trough_time'].append(blink_time)
            PB_Dictionary['stable_points'].append((0, 0))
            PB_Dictionary['amplitude'].append(0)
            PB_Dictionary['duration_opening'].append(0)
            PB_Dictionary['duration_closing'].append(0)
            PB_Dictionary['symmetry'].append(0)
            PB_Dictionary['blink_likelyhood'].append(0)
            PB_Dictionary['is_blink'].append(0)
            PB_Dictionary['duration_total'].append(0)
            PB_Dictionary['opening_velocity'].append(0)
            PB_Dictionary['closing_velocity'].append(0)
            PB_Dictionary['blink_validation'].append('FN')
    return PB_Dictionary

# Caclulates the final perfomance metrics of the code
def calculate_performance_from_dict(PB_Dictionary, total_blinks, global_confusion):
    validation = PB_Dictionary["blink_validation"]
    tp = validation.count("TP")
    fp = validation.count("FP")
    fn = validation.count("FN")
    tn = total_blinks - (tp + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    accuracy = (tp + tn) / total_blinks if total_blinks > 0 else 0
    global_confusion += np.array([[tp, fp], [fn, tn]])
    return precision, recall, accuracy, global_confusion


# Graphing Functions

In [ ]:
# Plot the signal and the blinks in order to ensure the system is working correctly
def plot_signal(data, time_column, signal, bandpassed_signal, PB_Dictionary, blink_times, channel, table, title, output_folder, SaveOutput, FileLabeled):
    save_folder = os.path.join(output_folder, "MainData", f"{table}_{channel}")
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    fig1 = plt.figure(figsize=(12, 4))
    plt.plot(data[time_column], data[channel], label=f'Original Signal ({channel})', color='blue', alpha=0.7)
    plt.title('Full Raw Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.legend()
    if SaveOutput:
        plt.savefig(os.path.join(save_folder, "full_raw_waveform.jpg"))
    plt.show()

    fig2 = plt.figure(figsize=(12, 4))
    plt.plot(data[time_column], scale_signal(bandpassed_signal), label='Bandpassed Signal', color='orange', alpha=0.8)
    plt.title('Full Filtered Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (Filtered)')
    plt.grid(True)
    plt.legend()
    if SaveOutput:
        plt.savefig(os.path.join(save_folder, "full_filtered_waveform.jpg"))
    plt.show()

    fig3 = plt.figure(figsize=(12, 4))
    plt.plot(data[time_column], signal, label='Noise-Reduced Signal', color='black')

    if PB_Dictionary.get("stable_points"):
        sp = [p for p in PB_Dictionary["stable_points"] if p is not None]
        xs = [x for pair in sp for x in pair]
        ys = [0] * len(xs)
        plt.scatter(xs, ys, color='purple', marker='x',
                    label='Stable Points', zorder=10)
    if FileLabeled:
        tp_idx = [i for i, v in enumerate(PB_Dictionary["blink_validation"]) if v == "TP"]
        fp_idx = [i for i, v in enumerate(PB_Dictionary["blink_validation"]) if v == "FP"]
        fn_idx = [i for i, v in enumerate(PB_Dictionary["blink_validation"]) if v == "FN"]

        if tp_idx:
            plt.scatter([PB_Dictionary["trough_time"][i] for i in tp_idx], [signal[i] for i in tp_idx], edgecolor='blue', facecolor='green', s=80, label='True Positives', zorder=10)
        if fp_idx:
            plt.scatter([PB_Dictionary["trough_time"][i] for i in fp_idx], [signal[i] for i in fp_idx], edgecolor='blue', facecolor='red', s=80, label='False Positives', zorder=10)
        if fn_idx:
            plt.scatter([PB_Dictionary["trough_time"][i] for i in fn_idx], [signal[i] for i in fn_idx], edgecolor='blue', facecolor='blue', s=80, label='False Negatives', zorder=10)
    else:
        blink_idx = [i for i, b in enumerate(PB_Dictionary["is_blink"]) if b == 1]
        not_idx = [i for i, b in enumerate(PB_Dictionary["is_blink"]) if b == 0]

        if blink_idx:
            plt.scatter([PB_Dictionary["trough_time"][i] for i in blink_idx], [signal[i] for i in blink_idx], edgecolor='blue', facecolor='green', s=80, label='Likely Blink', zorder=10)
        if not_idx:
            plt.scatter([PB_Dictionary["trough_time"][i] for i in not_idx], [signal[i] for i in not_idx], edgecolor='black', facecolor='orange', s=80, label='Not A Blink Trough', zorder=10)

    if blink_times is not None and len(blink_times) > 0:
        for i, bt in enumerate(blink_times):
            plt.axvline(x=bt, color='green', linestyle='--', label='Known Blink' if i == 0 else "", zorder=1)

    plt.title(title)
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude (Noise-Reduced)')
    plt.grid(True)
    plt.legend()
    if SaveOutput:
        plt.savefig(os.path.join(save_folder, "full_simplified_waveform.jpg"))
    plt.show()

def analyze_blink_similarity(PB_Dictionary):
    tp_indices = [i for i, val in enumerate(PB_Dictionary["blink_validation"]) if val in ["TP", "FN", "FP"]]
    features = ["amplitude", "duration_total", "symmetry"]

    if not tp_indices:
        print("No true positives detected. Skipping analysis.")
        return None, pd.DataFrame(columns=["Feature", "Mean", "Std Dev", "Min", "Max", "Range"])

    feature_matrix = np.array([[PB_Dictionary[feature][i] for feature in features] for i in tp_indices])
    feature_matrix = np.nan_to_num(feature_matrix)

    if feature_matrix.ndim == 1:
        feature_matrix = feature_matrix.reshape(-1, len(features))

    stats = {"Feature": [], "Mean": [], "Std Dev": [], "Min": [], "Max": [], "Range": []}
    for i, feature in enumerate(features):
        column = feature_matrix[:, i]
        stats["Feature"].append(feature.capitalize())
        stats["Mean"].append(np.mean(column))
        stats["Std Dev"].append(np.std(column))
        stats["Min"].append(np.min(column))
        stats["Max"].append(np.max(column))
        stats["Range"].append(np.ptp(column))

    stats_df = pd.DataFrame(stats)

    print("\nCompare Blink Critera:")
    print(stats_df.to_string(index=False, float_format="{:.2f}".format))

    similarity_matrix = cosine_similarity(feature_matrix)

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    x = feature_matrix[:, 0]
    y = feature_matrix[:, 1]
    z = feature_matrix[:, 2]

    edge_colors = { "TP": "green", "FP": "red", "FN": "blue"}

    scatter = ax.scatter(x, y, z, c=z, cmap="viridis", edgecolors=[edge_colors[PB_Dictionary["blink_validation"][idx]] for idx in tp_indices], s=100, linewidth=2)

    fig.colorbar(scatter, ax=ax, label="Symmetry (Color Gradient)")
    ax.set_title("Blink Critera Similarity")
    ax.set_xlabel("Amplitude")
    ax.set_ylabel("Duration Total")
    ax.set_zlabel("Symmetry")

    plt.show()

    return similarity_matrix, stats_df

# Plot the confusion matrix as a heatmap
def plot_confusion_matrix(conf_matrix):
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Blink', 'Predicted Not Blink'], yticklabels=['Actual Blink', 'Actual Not Blink'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    tp, fp, fn, tn = conf_matrix[0, 0], conf_matrix[0, 1], conf_matrix[1, 0], conf_matrix[1, 1]
    final_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    final_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    final_accuracy = (tp + tn) / (tp + fp + fn + tn) if (tp + fp + fn + tn) > 0 else 0

    print("\nFinal Performance Metrics (Cumulative):")
    print(f"Precision: {final_precision:.2f}")
    print(f"Recall: {final_recall:.2f}")
    print(f"Accuracy: {final_accuracy:.2f}")